In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [15]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [16]:
import warnings 
warnings.filterwarnings("ignore")

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [18]:
# train_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/train.csv")
# test_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")
# train_data.shape, test_data.shape

In [19]:
# train_data.head()

In [20]:
# train_data = train_data.drop("id", axis = 1)

In [21]:
# train_data.corr()["FloodProbability"]

In [22]:
# X = train_data.drop("FloodProbability", axis = 1)
# y = train_data["FloodProbability"]

# test df

In [23]:
# test_data.drop("id", axis = 1, inplace = True)

In [24]:
# column_names = X.columns

In [25]:
# sc = StandardScaler()
# X = sc.fit_transform(X)
# test_data = sc.transform(test_data)

In [26]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

## Model training
documentation : https://www.kaggle.com/code/carlmcbrideellis/gpu-accelerated-shap-values-jane-street-example

In [27]:
# model_params = {
#     "n_estimators" : 1000,
#     "learning_rate" : 0.01,
#     "max_depth" : 10,
#     "device" : device
# }

In [28]:
# eval_set = [(X_val, y_val)]

In [29]:
# model_XGB = XGBRegressor(**model_params)
# model_XGB.fit(X_train, y_train, eval_set=eval_set, verbose = False)

## Experiments

In [30]:
# temp = train_data.copy()
# column_names

In [31]:
# temp.head()

In [32]:
# X = temp.drop("FloodProbability", axis = 1)
# y = temp["FloodProbability"]

In [33]:
# for i, column in enumerate(column_names):
#     if i == 0:
#         continue
#     X[f"sum_of_{i}"] = X[list(column_names[:i])].sum(axis = 1)

In [34]:
# X.head()

In [35]:
# test_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")
# test_data.drop("id", axis = 1, inplace = True)
# for i, column in enumerate(column_names):
#     if i == 0:
#         continue
#     test_data[f"sum_of_{i}"] = test_data[list(column_names[:i])].sum(axis = 1)

In [36]:
# X = sc.fit_transform(X)
# test_data = sc.transform(test_data)

In [37]:
# model_XGB.fit(X, y, verbose = False)

## ANN

In [38]:
train_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/train.csv")
test_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")

train_data = train_data.drop("id", axis = 1)
test_data = test_data.drop("id", axis = 1)

X = train_data.drop("FloodProbability", axis = 1)
y = train_data["FloodProbability"]

sc = StandardScaler()
X = sc.fit_transform(X)
test_data = sc.transform(test_data)

In [39]:
import torch
from torch import nn
from torch.nn import functional as F

class RegressionNet(nn.Module):
    def __init__(self, n_features):
        super(RegressionNet, self).__init__()
        self.hidden1 = nn.Linear(n_features, 2048)
        self.hidden2 = nn.Linear(2048, 1024)
        self.hidden3 = nn.Linear(1024, 512)
        self.hidden4 = nn.Linear(512, 264)
        self.hidden5 = nn.Linear(264, 128)
        self.hidden6 = nn.Linear(128, 64)
        self.hidden7 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
    
    def forward(self, x):
      # Pass through the first hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden1(x))
      x = F.dropout(x, p=0.2)

      # Pass through the second hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden2(x))
      x = F.dropout(x, p=0.2)

      # Pass through the third hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden3(x))
      x = F.dropout(x, p=0.2)

      # Pass through the fourth hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden4(x))
      x = F.dropout(x, p=0.2)

      # Pass through the fifth hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden5(x))
      x = F.dropout(x, p=0.2)

      # Pass through the sixth hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden6(x))
      x = F.dropout(x, p=0.2)

      # Pass through the seventh hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden7(x))
      x = F.dropout(x, p=0.2)

      # Final output layer without activation (linear regression)
      return self.output(x)

In [40]:
n_features = X.shape[1]
n_features

20

In [41]:
X = torch.from_numpy(X.astype(np.float32))
y = torch.tensor(y.values.astype(np.float32))
test_data = torch.from_numpy(test_data.astype(np.float32))

In [42]:
X.dtype

torch.float32

In [43]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]
        return x, y

In [44]:
dataset = CustomDataset(X, y)

In [45]:
batch_size = 4096
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [46]:
from tqdm import tqdm

model = RegressionNet(n_features)
model.to(device)

loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters())

loss_vals = []

# pb = tqdm(total = 50, desc = "Training")

model.train()

for epoch in tqdm(range(20)):
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)

        # Forward pass
        y_pred = model(data)
        loss = loss_fn(y_pred.squeeze(), target)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(data_loader)
    loss_vals.append(avg_loss)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")
#     pb.set_postfix(loss=avg_loss)
#     pb.update(1)

  5%|▌         | 1/20 [00:10<03:10, 10.03s/it]

Epoch: 1, Loss: 0.0108


 10%|█         | 2/20 [00:19<02:52,  9.60s/it]

Epoch: 2, Loss: 0.0028


 15%|█▌        | 3/20 [00:28<02:38,  9.34s/it]

...

Epoch: 18, Loss: 0.0005


 95%|█████████▌| 19/20 [03:05<00:09,  9.63s/it]

Epoch: 19, Loss: 0.0004


100%|██████████| 20/20 [03:14<00:00,  9.73s/it]

Epoch: 20, Loss: 0.0004


In [47]:
torch.save(model.state_dict(), f"ann_flood.pth")

# Submission

In [48]:
model = RegressionNet(20)
model.load_state_dict(torch.load("/kaggle/working/ann_flood.pth"))
model.to(device)
model.eval()

RegressionNet(
  (hidden1): Linear(in_features=20, out_features=2048, bias=True)
  (hidden2): Linear(in_features=2048, out_features=1024, bias=True)
  (hidden3): Linear(in_features=1024, out_features=512, bias=True)
  (hidden4): Linear(in_features=512, out_features=264, bias=True)
  (hidden5): Linear(in_features=264, out_features=128, bias=True)
  (hidden6): Linear(in_features=128, out_features=64, bias=True)
  (hidden7): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
)

In [50]:
class CustomDataset(Dataset):
    def __init__(self, features):
        self.features = features
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        return x


test_dataset = CustomDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=4096, shuffle=False, num_workers=2)

all_predictions = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        predictions = model(data)
        all_predictions.append(predictions.cpu().numpy())  # to CPU and convert to numpy array

In [51]:
all_predictions = np.concatenate(all_predictions)
all_predictions

array([[0.57029265],
       [0.45785567],
       [0.45145577],
       ...,
       [0.6257881 ],
       [0.5660674 ],
       [0.51191956]], dtype=float32)

In [53]:
test_data_id = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")["id"]

In [54]:
submission = pd.DataFrame(list(zip(test_data_id, all_predictions)), columns = ["id", "FloodProbability"])
submission.head()

,id,FloodProbability
0,1117957,[0.57029265]
1,1117958,[0.45785567]
2,1117959,[0.45145577]
3,1117960,[0.47201493]
4,1117961,[0.47287413]


In [56]:
submission["FloodProbability"] = submission["FloodProbability"].apply(lambda x : x[0])
submission.head()

,id,FloodProbability
0,1117957,0.570293
1,1117958,0.457856
2,1117959,0.451456
3,1117960,0.472015
4,1117961,0.472874


In [57]:
submission.to_csv("submission.csv", index = False)

# SHAP

In [ ]:
# import xgboost as xgb
# import shap
# shap.initjs()

In [ ]:
# dmat = xgb.DMatrix(X, y)
# bst  = xgb.train({"tree_method": "gpu_hist"}, dmat, 500)
# # Make sure GPU prediction is enabled
# bst.set_param({"predictor": "gpu_predictor"})

In [ ]:
# %time
# shap_values = bst.predict(dmat, pred_contribs=True)

In [ ]:
# shap.summary_plot(shap_values[:,:-1], X)